In [1]:
import pandas as pd 
import numpy as np 
import math
from kmodes.kmodes import KModes

In [129]:
%config IPCompleter.greedy=True 

In [2]:
data = pd.read_csv("dataset.csv", error_bad_lines = False)

In [5]:
data.head()

,Unnamed: 0,Unnamed: 0.1,ID,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,FBI Code,Year,Latitude,Longitude
0,0,0,4786321,2004-01-01 00:01:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,4.0,06,2004.0,41.734106,-87.563621
1,1,1,4676906,2003-03-01 00:00:00,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,True,9.0,26,2003.0,41.817229,-87.637328
2,2,2,4789749,2004-06-20 11:00:00,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,False,14.0,20,2004.0,41.915620,-87.694019
3,3,3,4789765,2004-12-30 20:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,OTHER,False,False,25.0,06,2004.0,41.919054,-87.752178
4,4,4,4677901,2003-05-01 01:00:00,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,22.0,06,2003.0,41.691785,-87.635116


In [6]:
data["Year"].value_counts()

2008.0    852053
2006.0    794684
2009.0    783900
2010.0    700691
2007.0    621848
2001.0    568517
2002.0    490879
2003.0    475913
2005.0    455811
2004.0    388205
2011.0    352066
2012.0    335670
2013.0    306703
2014.0    274527
2016.0    265462
2015.0    262995
2017.0     11357
Name: Year, dtype: int64

In [3]:
data['Date'] = pd.to_datetime(data['Date'])

In [4]:
data_dropped = data.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Latitude', 'Longitude', 'ID', 'FBI Code'], axis=1)

In [5]:
data_dropped.columns

Index(['Date', 'IUCR', 'Primary Type', 'Description', 'Location Description',
       'Arrest', 'Domestic', 'District', 'Year'],
      dtype='object')

In [6]:
data_addMonth = data_dropped.assign(Month = data_dropped["Date"].dt.month)

In [7]:
data_addDay = data_addMonth.assign(Day = data_addMonth["Date"].dt.day)

In [8]:
data_addDay.columns

Index(['Date', 'IUCR', 'Primary Type', 'Description', 'Location Description',
       'Arrest', 'Domestic', 'District', 'Year', 'Month', 'Day'],
      dtype='object')

In [9]:
data_addHour = data_addDay.assign(Hour = data_addDay["Date"].dt.hour)

In [10]:
data_addHour.columns

Index(['Date', 'IUCR', 'Primary Type', 'Description', 'Location Description',
       'Arrest', 'Domestic', 'District', 'Year', 'Month', 'Day', 'Hour'],
      dtype='object')

In [11]:
data_dropDate = data_addHour.drop(['Date'], axis=1)

In [12]:
finalDf = data_dropDate
finalDf

,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour
0,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,4.0,2004.0,1,1,0
1,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,True,9.0,2003.0,3,1,0
2,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,False,14.0,2004.0,6,20,11
3,0840,THEFT,FINANCIAL ID THEFT: OVER $300,OTHER,False,False,25.0,2004.0,12,30,20
4,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,22.0,2003.0,5,1,1
5,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,APARTMENT,False,False,10.0,2004.0,8,1,0
6,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,True,True,5.0,2001.0,1,1,11
7,5007,OTHER OFFENSE,OTHER WEAPONS VIOLATION,RESIDENCE PORCH/HALLWAY,False,False,22.0,2003.0,3,15,0
8,0890,THEFT,FROM BUILDING,RESIDENCE,False,False,18.0,2004.0,9,16,10
9,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,11.0,2003.0,1,1,0


In [50]:
myData = finalDf.to_csv('myData.csv')

In [13]:
def filterYear(year):
    df = finalDf[finalDf.Year==year]
    return df

In [18]:
year2001 = filterYear(2001)
year2002 = filterYear(2002)
year2003 = filterYear(2003)
year2004 = filterYear(2004)

In [21]:
def num_clusters(dataset):
    num_observations = len(dataset)
    value = math.sqrt(num_observations/2)
    return int(round(value))

In [55]:
year2002_sample = year2002.sample(250000)

In [53]:
len(year2002)

490879

In [31]:
num_clusters(year2004)

441

In [57]:
num_clusters(year2002_sample)

354

In [14]:
#5 clusters
km_5 = KModes(n_clusters=5, init = "Huang", n_init = 5, verbose=1)

In [15]:
#10 clusters
km_10 = KModes(n_clusters=10, init = "Huang", n_init = 5, verbose=1)

In [16]:
#25 clusters
km_25 = KModes(n_clusters=25, init = "Huang", n_init = 5, verbose=1)

In [17]:
#50 clusters
km_50 = KModes(n_clusters=50, init = "Huang", n_init = 5, verbose=1)

In [25]:
#100 clusters
km_100 = KModes(n_clusters=100, init = "Huang", n_init = 5, verbose=1)

## YEAR2001

In [162]:
year2001 = filterYear(2001)

In [163]:
year2001 = year2001.reset_index()

In [75]:
year2001.to_csv("year2001.csv")

In [73]:
year2001.head()

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour
0,6,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,True,True,5.0,2001.0,1,1,11
1,17,1753,OFFENSE INVOLVING CHILDREN,SEX ASSLT OF CHILD BY FAM MBR,RESIDENCE,True,False,8.0,2001.0,1,1,0
2,18,0840,THEFT,FINANCIAL ID THEFT: OVER $300,OTHER,False,False,9.0,2001.0,1,1,0
3,20,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,True,False,7.0,2001.0,6,16,8
4,25,1563,SEX OFFENSE,CRIMINAL SEXUAL ABUSE,RESIDENCE,False,False,15.0,2001.0,6,1,8


### 5 clusters

In [121]:
clusters_5 = km_5.fit_predict(year2001)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 69727, cost: 4074126.0
Run 1, iteration: 2/100, moves: 0, cost: 4074126.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 157023, cost: 3999731.0
Run 2, iteration: 2/100, moves: 6032, cost: 3999731.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 158964, cost: 4008336.0
Run 3, iteration: 2/100, moves: 44100, cost: 4008336.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 109772, cost: 4120841.0
Run 4, iteration: 2/100, moves: 2219, cost: 4120841.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 206651, cost: 4023302.0
Run 5, iteration: 2/100, moves: 87845, cost: 3962124.0
Run 5, iteration: 3/100, moves: 3173, cost: 3962124.

In [122]:
print(clusters_5)
print(len(clusters_5))
print(len(year2001))

[1 4 3 ..., 0 0 0]
568517
568517


In [123]:
km_5.labels_

array([1, 4, 3, ..., 0, 0, 0], dtype=uint8)

In [169]:
km_5.cluster_centroids_

array([['32', '0820', 'THEFT', '$500 AND UNDER', 'STREET', 'False',
        'False', '8.0', '2001.0', '10', '15', '22'],
       ['6', '0460', 'BATTERY', 'SIMPLE', 'STREET', 'False', 'False',
        '2.0', '2001.0', '3', '1', '19'],
       ['1551', '2027', 'NARCOTICS', 'POSS: CRACK', 'STREET', 'True',
        'False', '11.0', '2001.0', '2', '27', '20'],
       ['18', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', 'False',
        'False', '3.0', '2001.0', '11', '1', '14'],
       ['132', '2820', 'OTHER OFFENSE', 'TELEPHONE THREAT', 'RESIDENCE',
        'False', 'False', '6.0', '2001.0', '1', '9', '17']], 
      dtype='<U32')

In [194]:
year2001_5clusterCentroids = pd.DataFrame(km_5.cluster_centroids_)

In [195]:
year2001_5clusterCentroids

,0,1,2,3,4,5,6,7,8,9,10,11
0,32,0820,THEFT,$500 AND UNDER,STREET,False,False,8.0,2001.0,10,15,22
1,6,0460,BATTERY,SIMPLE,STREET,False,False,2.0,2001.0,3,1,19
2,1551,2027,NARCOTICS,POSS: CRACK,STREET,True,False,11.0,2001.0,2,27,20
3,18,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,3.0,2001.0,11,1,14
4,132,2820,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,False,6.0,2001.0,1,9,17


In [196]:
year2001_5clusterCentroids.columns = year2001.columns

In [197]:
year2001_5clusterCentroids

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour
0,32,0820,THEFT,$500 AND UNDER,STREET,False,False,8.0,2001.0,10,15,22
1,6,0460,BATTERY,SIMPLE,STREET,False,False,2.0,2001.0,3,1,19
2,1551,2027,NARCOTICS,POSS: CRACK,STREET,True,False,11.0,2001.0,2,27,20
3,18,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,3.0,2001.0,11,1,14
4,132,2820,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,False,6.0,2001.0,1,9,17


In [198]:
year2001_5clusterCentroids.to_csv("year2001_5clusterCentroids.csv")

In [126]:
year2001_5clusters = pd.DataFrame(clusters_5)

In [128]:
year2001_5clusters.columns = ['clusters_5']

In [130]:
year2001_5clusters = year2001_5clusters.reset_index()

In [132]:
year2001_5clusters['clusters_5'].value_counts()

0    192179
1    158954
2     98370
3     63621
4     55393
Name: clusters_5, dtype: int64

In [133]:
year2001New5 = pd.concat([year2001, year2001_5clusters], axis = 1)

In [135]:
year2001New5.to_csv("year2001_5clusters.csv")

### 10 clusters

In [137]:
clusters_10 = km_10.fit_predict(year2001)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 155244, cost: 3770579.0
Run 1, iteration: 2/100, moves: 19998, cost: 3770578.0
Run 1, iteration: 3/100, moves: 0, cost: 3770578.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 173764, cost: 3670228.0
Run 2, iteration: 2/100, moves: 60909, cost: 3667974.0
Run 2, iteration: 3/100, moves: 3770, cost: 3667974.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 120852, cost: 3674605.0
Run 3, iteration: 2/100, moves: 16629, cost: 3674605.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 123070, cost: 3738500.0
Run 4, iteration: 2/100, moves: 47828, cost: 3738499.0
Run 4, iteration: 3/100, moves: 0, cost: 3738499.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...


In [138]:
print(clusters_10)
print(len(clusters_10))
print(len(year2001))

[0 4 8 ..., 8 0 0]
568517
568517


In [139]:
km_10.labels_

array([0, 4, 8, ..., 8, 0, 0], dtype=uint8)

In [140]:
year2001_10clusters = pd.DataFrame(clusters_10)

In [142]:
year2001_10clusters.columns = ['clusters_10']

In [143]:
year2001_10clusters = year2001_10clusters.reset_index()

In [145]:
year2001_10clusters['clusters_10'].value_counts()

0    88396
8    86243
4    70162
1    67594
3    63266
2    55372
7    43964
9    40865
5    29953
6    22702
Name: clusters_10, dtype: int64

In [146]:
year2001New10 = pd.concat([year2001, year2001_10clusters], axis = 1)

In [149]:
year2001New10.to_csv("year2001_10clusters.csv")

In [185]:
km_10.cluster_centroids_

array([['6', '0810', 'THEFT', 'OVER $500', 'STREET', 'False', 'False',
        '5.0', '2001.0', '6', '29', '22'],
       ['32', '0460', 'BATTERY', 'SIMPLE', 'RESIDENCE', 'False', 'True',
        '25.0', '2001.0', '7', '21', '21'],
       ['1078', '0560', 'ASSAULT', 'SIMPLE', 'RESIDENCE', 'False', 'False',
        '7.0', '2001.0', '2', '6', '17'],
       ['132', '0460', 'BATTERY', 'SIMPLE', 'RESIDENCE', 'False', 'False',
        '18.0', '2001.0', '12', '4', '9'],
       ['17', '2027', 'NARCOTICS', 'POSS: CRACK', 'STREET', 'True',
        'False', '2.0', '2001.0', '2', '26', '19'],
       ['20', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS',
        'STREET', 'True', 'False', '7.0', '2001.0', '4', '18', '20'],
       ['382', '0560', 'ASSAULT', 'SIMPLE', 'STREET', 'False', 'False',
        '14.0', '2001.0', '11', '20', '16'],
       ['1193', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', 'False',
        'False', '3.0', '2001.0', '5', '6', '0'],
       ['335', '0820', 'THEFT', '

In [186]:
year2001_10clusterCentroids = pd.DataFrame(km_10.cluster_centroids_)

In [188]:
year2001_10clusterCentroids.columns = year2001.columns

In [191]:
year2001_10clusterCentroids

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour
0,6,0810,THEFT,OVER $500,STREET,False,False,5.0,2001.0,6,29,22
1,32,0460,BATTERY,SIMPLE,RESIDENCE,False,True,25.0,2001.0,7,21,21
2,1078,0560,ASSAULT,SIMPLE,RESIDENCE,False,False,7.0,2001.0,2,6,17
3,132,0460,BATTERY,SIMPLE,RESIDENCE,False,False,18.0,2001.0,12,4,9
4,17,2027,NARCOTICS,POSS: CRACK,STREET,True,False,2.0,2001.0,2,26,19
5,20,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,STREET,True,False,7.0,2001.0,4,18,20
6,382,0560,ASSAULT,SIMPLE,STREET,False,False,14.0,2001.0,11,20,16
7,1193,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,3.0,2001.0,5,6,0
8,335,0820,THEFT,$500 AND UNDER,STREET,False,False,8.0,2001.0,1,19,15
9,118,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,10.0,2001.0,10,1,21


In [199]:
year2001_10clusterCentroids.to_csv("year2001_10clusterCentroids.csv")

### 25 clusters

In [164]:
clusters2001_25 = km_25.fit_predict(year2001)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 134968, cost: 3376709.0
Run 1, iteration: 2/100, moves: 53954, cost: 3344661.0
Run 1, iteration: 3/100, moves: 9329, cost: 3342911.0
Run 1, iteration: 4/100, moves: 250, cost: 3342911.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 106601, cost: 3437147.0
Run 2, iteration: 2/100, moves: 14181, cost: 3437147.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 108098, cost: 3464905.0
Run 3, iteration: 2/100, moves: 47476, cost: 3460681.0
Run 3, iteration: 3/100, moves: 1915, cost: 3460681.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 151996, cost: 3400527.0
Run 4, iteration: 2/100, moves: 8417, cost: 3400527.0
Init: initializing centroids
Init: initializing clusters
Starting iterations

In [179]:
year2001_25clusters = pd.DataFrame(clusters2001_25)

In [180]:
year2001_25clusters.columns = ['clusters_25']

In [181]:
year2001New25 = pd.concat([year2001, year2001_25clusters], axis = 1)

In [182]:
year2001New25.to_csv("year2001_25clusters.csv")

In [183]:
year2001_25clusterCentroids = pd.DataFrame(km_25.cluster_centroids_)

In [184]:
year2001_25clusterCentroids.to_csv("year2001_25clusterCentroids.csv")

### 50 clusters

In [55]:
clusters = km.fit_predict(year2001)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 42531, cost: 2655475.0
Run 1, iteration: 2/100, moves: 2859, cost: 2655475.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 84123, cost: 2631513.0
Run 2, iteration: 2/100, moves: 4326, cost: 2631513.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 73502, cost: 2661354.0
Run 3, iteration: 2/100, moves: 16300, cost: 2658658.0
Run 3, iteration: 3/100, moves: 3625, cost: 2657887.0
Run 3, iteration: 4/100, moves: 783, cost: 2657887.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 80817, cost: 2581660.0
Run 4, iteration: 2/100, moves: 6855, cost: 2581660.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 78343, cost: 2603444.0
R

In [59]:
print(clusters)
print(len(clusters))
print(len(year2001))

[10  4 26 ...,  0 30 13]
568517
568517


In [61]:
km.labels_

array([10,  4, 26, ...,  0, 30, 13], dtype=uint8)

In [60]:
km.cluster_centroids_

array([['0560', 'ASSAULT', 'SIMPLE', 'RESIDENCE', 'False', 'False', '18.0',
        '2001.0', '7', '15', '23'],
       ['0820', 'THEFT', '$500 AND UNDER', 'RESIDENCE', 'False', 'False',
        '18.0', '2001.0', '12', '9', '18'],
       ['0460', 'BATTERY', 'SIMPLE', 'OTHER', 'False', 'False', '12.0',
        '2001.0', '12', '28', '17'],
       ['0460', 'BATTERY', 'SIMPLE', 'RESIDENCE', 'False', 'False', '16.0',
        '2001.0', '3', '29', '22'],
       ['1330', 'CRIMINAL TRESPASS', 'TO LAND', 'RESIDENCE', 'True',
        'False', '2.0', '2001.0', '3', '26', '23'],
       ['0460', 'BATTERY', 'SIMPLE', 'STREET', 'True', 'False', '12.0',
        '2001.0', '1', '13', '1'],
       ['031A', 'ROBBERY', 'ARMED: HANDGUN', 'STREET', 'False', 'False',
        '15.0', '2001.0', '11', '15', '22'],
       ['2820', 'OTHER OFFENSE', 'TELEPHONE THREAT', 'RESIDENCE', 'False',
        'False', '16.0', '2001.0', '1', '23', '8'],
       ['0910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', 'False',
    

In [62]:
year2001_50clusters = pd.DataFrame(clusters)

In [63]:
year2001_50clusters

,0
0,10
1,4
2,26
3,42
4,7
5,0
6,42
7,26
8,30
9,1


In [65]:
year2001_50clusters.columns = ['clusters_50']

In [69]:
year2001_50clusters['clusters_50'].value_counts()

18    37927
1     27775
10    26610
15    25760
8     24958
3     24190
0     24007
4     18182
26    18134
37    17473
13    17253
6     16839
2     15933
7     15894
5     15608
40    14564
32    14166
23    13478
12    12872
41    12753
30    12628
14    10531
9      8831
47     8737
45     8323
20     7822
27     7798
16     7402
36     7201
24     7162
17     6942
29     5599
22     5560
38     5316
25     5074
42     5056
33     4871
43     4718
19     4706
21     4687
34     4656
44     4434
31     4391
46     3832
28     3409
11     3263
35     3169
49     2942
39     2549
48     2532
Name: clusters_50, dtype: int64

In [66]:
year2001_50clusters = year2001_50clusters.reset_index()

In [67]:
year2001New = pd.concat([year2001, year2001_50clusters], axis = 1)

In [68]:
year2001New

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour,index,clusters_50
0,6,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,True,True,5.0,2001.0,1,1,11,0,10
1,17,1753,OFFENSE INVOLVING CHILDREN,SEX ASSLT OF CHILD BY FAM MBR,RESIDENCE,True,False,8.0,2001.0,1,1,0,1,4
2,18,0840,THEFT,FINANCIAL ID THEFT: OVER $300,OTHER,False,False,9.0,2001.0,1,1,0,2,26
3,20,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,True,False,7.0,2001.0,6,16,8,3,42
4,25,1563,SEX OFFENSE,CRIMINAL SEXUAL ABUSE,RESIDENCE,False,False,15.0,2001.0,6,1,8,4,7
5,32,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,True,14.0,2001.0,7,25,10,5,0
6,33,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,7.0,2001.0,6,23,20,6,42
7,50,0842,THEFT,AGG: FINANCIAL ID THEFT,OTHER,False,False,8.0,2001.0,1,1,0,7,26
8,63,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,False,4.0,2001.0,5,1,0,8,30
9,118,0842,THEFT,AGG: FINANCIAL ID THEFT,RESIDENCE,False,False,7.0,2001.0,10,1,0,9,1


In [71]:
year2001New.to_csv("year2001_50clusters.csv")

In [201]:
km.cluster_centroids_

array([['41', '0486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE',
        'True', 'False', '18.0', '2004.0', '6', '1', '17'],
       ['48', '0610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE', 'True',
        'False', '5.0', '2004.0', '8', '8', '19'],
       ['5', '0560', 'ASSAULT', 'SIMPLE', 'STREET', 'False', 'False',
        '11.0', '2004.0', '8', '21', '21'],
       ['517', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', 'True',
        'False', '16.0', '2004.0', '7', '29', '15'],
       ['74', '0460', 'BATTERY', 'SIMPLE', 'SIDEWALK', 'False', 'False',
        '22.0', '2004.0', '7', '16', '21'],
       ['119', '0910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET',
        'False', 'False', '14.0', '2004.0', '12', '2', '23'],
       ['86', '0430', 'BATTERY', 'AGGRAVATED: OTHER DANG WEAPON', 'STREET',
        'False', 'False', '9.0', '2004.0', '5', '21', '20'],
       ['2583', '0486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'SIDEWALK',
        'True', 'False', '3.0', '2004.0', 

In [202]:
year2001_50clusterCentroids = pd.DataFrame(km.cluster_centroids_)

In [204]:
year2001_50clusterCentroids.columns = year2001.columns

In [205]:
year2001_50clusterCentroids

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour
0,41,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,True,False,18.0,2004.0,6,1,17
1,48,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,True,False,5.0,2004.0,8,8,19
2,5,0560,ASSAULT,SIMPLE,STREET,False,False,11.0,2004.0,8,21,21
3,517,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,True,False,16.0,2004.0,7,29,15
4,74,0460,BATTERY,SIMPLE,SIDEWALK,False,False,22.0,2004.0,7,16,21
5,119,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,14.0,2004.0,12,2,23
6,86,0430,BATTERY,AGGRAVATED: OTHER DANG WEAPON,STREET,False,False,9.0,2004.0,5,21,20
7,2583,0486,BATTERY,DOMESTIC BATTERY SIMPLE,SIDEWALK,True,False,3.0,2004.0,5,26,20
8,162,0860,THEFT,RETAIL THEFT,GROCERY FOOD STORE,True,False,1.0,2004.0,8,4,12
9,176,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,STREET,True,False,7.0,2004.0,1,2,12


In [206]:
year2001_50clusterCentroids.to_csv("year2001_50clusterCentroids.csv")

### 100 clusters

In [151]:
clusters_100 = km_100.fit_predict(year2001)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 93295, cost: 3031846.0
Run 1, iteration: 2/100, moves: 2805, cost: 3031846.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 107885, cost: 2994534.0
Run 2, iteration: 2/100, moves: 11593, cost: 2994039.0
Run 2, iteration: 3/100, moves: 1263, cost: 2994039.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 103415, cost: 3021215.0
Run 3, iteration: 2/100, moves: 4952, cost: 3021207.0
Run 3, iteration: 3/100, moves: 0, cost: 3021207.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 57143, cost: 3005338.0
Run 4, iteration: 2/100, moves: 6387, cost: 3005336.0
Run 4, iteration: 3/100, moves: 0, cost: 3005336.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5

In [153]:
print(clusters_100)
print(len(clusters_100))
print(len(year2001))

[ 9 27  1 ..., 41  1 31]
568517
568517


In [154]:
km_100.labels_

array([ 9, 27,  1, ..., 41,  1, 31], dtype=uint8)

In [155]:
year2001_100clusters = pd.DataFrame(clusters_100)

In [156]:
year2001_100clusters.columns = ['clusters_100']

In [157]:
year2001_100clusters = year2001_100clusters.reset_index()

In [158]:
year2001_100clusters['clusters_100'].value_counts()

5     24699
21    20715
27    19010
70    18498
3     18445
2     17332
7     17119
12    16448
29    16041
57    13523
4     11645
10    11202
9     10067
11     9949
88     9670
31     9460
40     9334
32     9327
15     9253
17     9044
6      8375
92     8088
0      7934
38     7737
25     7177
8      7157
35     6544
20     6463
14     6451
24     6037
      ...  
47     2549
65     2517
76     2490
48     2426
86     2365
64     2234
41     2220
83     2152
69     2147
78     2141
89     2137
60     2099
71     2018
74     2004
61     1905
1      1854
99     1753
90     1752
54     1716
81     1689
94     1675
95     1535
97     1443
80     1236
82     1225
72     1140
66      774
67      763
68      722
96      708
Name: clusters_100, dtype: int64

In [159]:
year2001New100 = pd.concat([year2001, year2001_100clusters], axis = 1)

In [161]:
year2001New100.to_csv("year2001_100clusters.csv")

In [207]:
km_100.cluster_centroids_

array([['252', '2825', 'OTHER OFFENSE', ..., '9', '10', '18'],
       ['18', '0840', 'THEFT', ..., '1', '1', '9'],
       ['1976', '0460', 'BATTERY', ..., '3', '20', '23'],
       ..., 
       ['7920', '0460', 'BATTERY', ..., '2', '10', '18'],
       ['2845', '1320', 'CRIMINAL DAMAGE', ..., '11', '1', '22'],
       ['2711', '1310', 'CRIMINAL DAMAGE', ..., '12', '17', '1']], 
      dtype='<U32')

In [208]:
year2001_100clusterCentroids = pd.DataFrame(km_100.cluster_centroids_)

In [210]:
year2001_100clusterCentroids.columns = year2001.columns

In [211]:
year2001_100clusterCentroids

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour
0,252,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,OTHER,False,False,3.0,2001.0,9,10,18
1,18,0840,THEFT,FINANCIAL ID THEFT: OVER $300,APARTMENT,False,True,25.0,2001.0,1,1,9
2,1976,0460,BATTERY,SIMPLE,RESIDENCE,False,True,5.0,2001.0,3,20,23
3,519,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,2.0,2001.0,5,31,17
4,132,0460,BATTERY,SIMPLE,RESIDENCE,False,False,7.0,2001.0,2,28,14
5,854,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,25.0,2001.0,12,23,20
6,335,1210,DECEPTIVE PRACTICE,THEFT OF LABOR/SERVICES,RESIDENCE,False,False,6.0,2001.0,1,19,15
7,1438,0820,THEFT,$500 AND UNDER,STREET,False,False,11.0,2001.0,10,29,10
8,751,0460,BATTERY,SIMPLE,SIDEWALK,False,False,15.0,2001.0,2,16,9
9,6,0810,THEFT,OVER $500,RESIDENCE,False,False,17.0,2001.0,1,20,11


In [212]:
year2001_100clusterCentroids.to_csv("year2001_100clusterCentroids.csv")

### YEAR2002

In [145]:
year2002 = filterYear(2002)

In [146]:
year2002 = year2002.reset_index()

In [147]:
year2002.head()

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour
0,19,0266,CRIM SEXUAL ASSAULT,PREDATORY,APARTMENT,False,False,14.0,2002.0,6,14,16
1,42,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,10.0,2002.0,7,1,0
2,72,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,False,25.0,2002.0,1,1,0
3,73,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,HOTEL/MOTEL,False,True,18.0,2002.0,1,1,0
4,77,0840,THEFT,FINANCIAL ID THEFT: OVER $300,BANK,False,False,4.0,2002.0,1,9,12


In [72]:
year2002.to_csv("year2002.csv")

In [26]:
clusters2002_5 = km_5.fit_predict(year2002)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 62933, cost: 3584926.0
Run 1, iteration: 2/100, moves: 4227, cost: 3584926.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 96609, cost: 3532830.0
Run 2, iteration: 2/100, moves: 43370, cost: 3526125.0
Run 2, iteration: 3/100, moves: 1568, cost: 3526125.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 126807, cost: 3569426.0
Run 3, iteration: 2/100, moves: 29884, cost: 3569426.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 138889, cost: 3544870.0
Run 4, iteration: 2/100, moves: 54750, cost: 3535272.0
Run 4, iteration: 3/100, moves: 10695, cost: 3530210.0
Run 4, iteration: 4/100, moves: 1378, cost: 3530210.0
Init: initializing centroids
Init: initializing clusters
Starting iterations

In [27]:
print(clusters2002_5)
print(len(clusters2002_5))
print(len(year2002))

[3 0 3 ..., 2 0 0]
490879
490879


In [29]:
km_5.labels_

array([3, 0, 3, ..., 2, 0, 0], dtype=uint8)

In [30]:
km_5.cluster_centroids_

array([['42', '0810', 'THEFT', 'OVER $500', 'STREET', 'False', 'False',
        '6.0', '2002.0', '7', '10', '21'],
       ['356', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS',
        'STREET', 'True', 'False', '11.0', '2002.0', '10', '22', '20'],
       ['92', '0820', 'THEFT', '$500 AND UNDER', 'STREET', 'False',
        'False', '11.0', '2002.0', '12', '11', '22'],
       ['19', '0486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE',
        'False', 'True', '25.0', '2002.0', '6', '7', '23'],
       ['545', '0460', 'BATTERY', 'SIMPLE', 'STREET', 'False', 'False',
        '2.0', '2002.0', '4', '30', '21']], 
      dtype='<U32')

In [31]:
year2002_5clusters = pd.DataFrame(clusters2002_5)

In [33]:
year2002_5clusters.columns = ['clusters_5']

In [35]:
year2002_5clusters

,clusters_5
0,3
1,0
2,3
3,3
4,0
5,4
6,2
7,3
8,0
9,0


In [37]:
year2002_5clusters['clusters_5'].value_counts()

0    167676
1    100627
4     86674
2     81212
3     54690
Name: clusters_5, dtype: int64

In [38]:
year2002New5 = pd.concat([year2002, year2002_5clusters], axis = 1)

In [39]:
year2002New5

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour,index,clusters_5
0,19,0266,CRIM SEXUAL ASSAULT,PREDATORY,APARTMENT,False,False,14.0,2002.0,6,14,16,0,3
1,42,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,10.0,2002.0,7,1,0,1,0
2,72,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,False,25.0,2002.0,1,1,0,2,3
3,73,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,HOTEL/MOTEL,False,True,18.0,2002.0,1,1,0,3,3
4,77,0840,THEFT,FINANCIAL ID THEFT: OVER $300,BANK,False,False,4.0,2002.0,1,9,12,4,0
5,78,1562,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,APARTMENT,False,False,2.0,2002.0,1,1,0,5,4
6,92,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,10.0,2002.0,12,6,7,6,2
7,98,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,True,25.0,2002.0,1,1,1,7,3
8,109,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,False,False,22.0,2002.0,7,21,23,8,0
9,114,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,False,False,15.0,2002.0,1,1,0,9,0


In [40]:
year2002New5.to_csv("year2002_5clusters.csv")

In [41]:
year2002_5clusterCentroids = pd.DataFrame(km_5.cluster_centroids_)

In [43]:
year2002_5clusterCentroids.columns = year2002.columns

In [44]:
year2002_5clusterCentroids

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour
0,42,0810,THEFT,OVER $500,STREET,False,False,6.0,2002.0,7,10,21
1,356,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,STREET,True,False,11.0,2002.0,10,22,20
2,92,0820,THEFT,$500 AND UNDER,STREET,False,False,11.0,2002.0,12,11,22
3,19,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,25.0,2002.0,6,7,23
4,545,0460,BATTERY,SIMPLE,STREET,False,False,2.0,2002.0,4,30,21


In [45]:
year2002_5clusterCentroids.to_csv("year2002_5clusterCentroids.csv")

In [46]:
clusters2002_10 = km_10.fit_predict(year2002)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 62975, cost: 3290246.0
Run 1, iteration: 2/100, moves: 15641, cost: 3286332.0
Run 1, iteration: 3/100, moves: 2053, cost: 3286332.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 146208, cost: 3379475.0
Run 2, iteration: 2/100, moves: 7601, cost: 3379474.0
Run 2, iteration: 3/100, moves: 0, cost: 3379474.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 60106, cost: 3394836.0
Run 3, iteration: 2/100, moves: 0, cost: 3394836.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 87346, cost: 3292842.0
Run 4, iteration: 2/100, moves: 9511, cost: 3292842.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 92758, cost: 3359243.0
Run 5

In [47]:
print(clusters2002_10)
print(len(clusters2002_10))
print(len(year2002))

[0 5 5 ..., 5 4 4]
490879
490879


In [49]:
km_10.labels_

array([0, 5, 5, ..., 5, 4, 4], dtype=uint8)

In [51]:
km_10.cluster_centroids_

array([['19', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', 'False',
        'False', '8.0', '2002.0', '7', '13', '16'],
       ['114', '0460', 'BATTERY', 'SIMPLE', 'SIDEWALK', 'False', 'False',
        '15.0', '2002.0', '8', '3', '10'],
       ['73', '0486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'STREET',
        'False', 'True', '8.0', '2002.0', '5', '8', '14'],
       ['463', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS',
        'STREET', 'True', 'False', '11.0', '2002.0', '5', '30', '18'],
       ['77', '0820', 'THEFT', '$500 AND UNDER', 'STREET', 'False',
        'False', '2.0', '2002.0', '8', '14', '20'],
       ['127', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE',
        'False', 'False', '14.0', '2002.0', '2', '2', '0'],
       ['236', '0610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE', 'False',
        'False', '3.0', '2002.0', '8', '7', '17'],
       ['168', '2825', 'OTHER OFFENSE', 'HARASSMENT BY TELEPHONE',
        'RESIDENCE', 'False', 'False', '18.0',

In [52]:
year2002_10clusters = pd.DataFrame(clusters2002_10)

In [54]:
year2002_10clusters.columns = ['clusters_10']

In [56]:
year2002_10clusters = year2002_10clusters.reset_index()

In [60]:
year2002_10clusters['clusters_10'].value_counts()

0    90374
4    87753
3    80152
1    78892
5    40465
6    29279
7    25957
9    21499
2    20621
8    15887
Name: clusters_10, dtype: int64

In [61]:
year2002New10 = pd.concat([year2002, year2002_10clusters], axis = 1)

In [63]:
year2002New10.to_csv("year2002_10clusters.csv")

In [64]:
year2002_10clusterCentroids = pd.DataFrame(km_10.cluster_centroids_)

In [66]:
year2002_10clusterCentroids.columns = year2002.columns

In [68]:
year2002_10clusterCentroids.to_csv("year2002_10clusterCentroids.csv")

### 25 clusters

In [148]:
clusters2002_25 = km_25.fit_predict(year2002)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 43645, cost: 3043639.0
Run 1, iteration: 2/100, moves: 0, cost: 3043639.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 117387, cost: 2978566.0
Run 2, iteration: 2/100, moves: 15510, cost: 2978565.0
Run 2, iteration: 3/100, moves: 0, cost: 2978565.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 112008, cost: 2964420.0
Run 3, iteration: 2/100, moves: 16091, cost: 2964419.0
Run 3, iteration: 3/100, moves: 0, cost: 2964419.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 67695, cost: 2993243.0
Run 4, iteration: 2/100, moves: 14054, cost: 2991399.0
Run 4, iteration: 3/100, moves: 1024, cost: 2991399.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5,

In [150]:
year2002_25clusters = pd.DataFrame(clusters2002_25)

In [151]:
year2002_25clusters.columns = ['clusters_25']

In [153]:
year2002New25 = pd.concat([year2002, year2002_25clusters], axis = 1)

In [155]:
year2002New25.to_csv("year2002_25clusters.csv")

In [156]:
year2002_25clusterCentroids = pd.DataFrame(km_25.cluster_centroids_)

In [157]:
year2002_25clusterCentroids.columns = year2002.columns

In [159]:
year2002_25clusterCentroids.to_csv("year2002_25clusterCentroids.csv")

### 50 clusters

In [71]:
clusters2002_50 = km_50.fit_predict(year2002)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 66998, cost: 2808720.0
Run 1, iteration: 2/100, moves: 2752, cost: 2808720.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 69648, cost: 2831870.0
Run 2, iteration: 2/100, moves: 4981, cost: 2831869.0
Run 2, iteration: 3/100, moves: 0, cost: 2831869.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 52517, cost: 2826792.0
Run 3, iteration: 2/100, moves: 13753, cost: 2826792.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 108031, cost: 2780242.0
Run 4, iteration: 2/100, moves: 36236, cost: 2772063.0
Run 4, iteration: 3/100, moves: 2165, cost: 2772063.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 43041, cost: 2874937.0
R

In [72]:
print(clusters2002_50)
print(len(clusters2002_50))
print(len(year2002))

[ 5  6 16 ...,  4  6  3]
490879
490879


In [73]:
km_50.labels_

array([ 5,  6, 16, ...,  4,  6,  3], dtype=uint8)

In [74]:
km_50.cluster_centroids_

array([['1332', '2027', 'NARCOTICS', 'POSS: CRACK', 'SIDEWALK', 'True',
        'False', '15.0', '2002.0', '5', '23', '19'],
       ['352', '0486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE',
        'False', 'False', '9.0', '2002.0', '11', '5', '21'],
       ['265', '0620', 'BURGLARY', 'UNLAWFUL ENTRY', 'RESIDENCE', 'False',
        'False', '19.0', '2002.0', '5', '13', '9'],
       ['78', '0820', 'THEFT', '$500 AND UNDER', 'STREET', 'False',
        'False', '2.0', '2002.0', '8', '3', '12'],
       ['92', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE',
        'False', 'False', '8.0', '2002.0', '12', '15', '0'],
       ['19', '2820', 'OTHER OFFENSE', 'TELEPHONE THREAT', 'APARTMENT',
        'False', 'False', '7.0', '2002.0', '2', '18', '16'],
       ['42', '0810', 'THEFT', 'OVER $500', 'RESIDENCE', 'False', 'False',
        '10.0', '2002.0', '7', '7', '9'],
       ['2868', '0560', 'ASSAULT', 'SIMPLE', 'STREET', 'False', 'False',
        '2.0', '2002.0', '5', '30', '13']

In [76]:
year2002_50clusters = pd.DataFrame(clusters2002_50)

In [77]:
year2002_50clusters

,0
0,5
1,6
2,16
3,34
4,21
5,3
6,4
7,16
8,26
9,4


In [78]:
year2002_50clusters.columns = ['clusters_50']

In [79]:
year2002_50clusters = year2002_50clusters.reset_index()

In [82]:
year2002New50 = pd.concat([year2002, year2002_50clusters], axis = 1)

In [83]:
year2002New50

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour,index,clusters_50
0,19,0266,CRIM SEXUAL ASSAULT,PREDATORY,APARTMENT,False,False,14.0,2002.0,6,14,16,0,5
1,42,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,10.0,2002.0,7,1,0,1,6
2,72,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,False,25.0,2002.0,1,1,0,2,16
3,73,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,HOTEL/MOTEL,False,True,18.0,2002.0,1,1,0,3,34
4,77,0840,THEFT,FINANCIAL ID THEFT: OVER $300,BANK,False,False,4.0,2002.0,1,9,12,4,21
5,78,1562,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,APARTMENT,False,False,2.0,2002.0,1,1,0,5,3
6,92,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,10.0,2002.0,12,6,7,6,4
7,98,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,True,25.0,2002.0,1,1,1,7,16
8,109,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,False,False,22.0,2002.0,7,21,23,8,26
9,114,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,False,False,15.0,2002.0,1,1,0,9,4


In [84]:
year2002New50.to_csv("year2002_50clusters.csv")

In [85]:
year2002_50clusterCentroids = pd.DataFrame(km_50.cluster_centroids_)

In [86]:
year2002_50clusterCentroids.columns = year2002.columns

In [87]:
year2002_50clusterCentroids

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour
0,1332,2027,NARCOTICS,POSS: CRACK,SIDEWALK,True,False,15.0,2002.0,5,23,19
1,352,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,False,9.0,2002.0,11,5,21
2,265,0620,BURGLARY,UNLAWFUL ENTRY,RESIDENCE,False,False,19.0,2002.0,5,13,9
3,78,0820,THEFT,$500 AND UNDER,STREET,False,False,2.0,2002.0,8,3,12
4,92,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,8.0,2002.0,12,15,0
5,19,2820,OTHER OFFENSE,TELEPHONE THREAT,APARTMENT,False,False,7.0,2002.0,2,18,16
6,42,0810,THEFT,OVER $500,RESIDENCE,False,False,10.0,2002.0,7,7,9
7,2868,0560,ASSAULT,SIMPLE,STREET,False,False,2.0,2002.0,5,30,13
8,2170,2024,NARCOTICS,POSS: HEROIN(WHITE),STREET,True,False,11.0,2002.0,12,17,11
9,441,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,OTHER,False,False,8.0,2002.0,6,21,9


In [88]:
year2002_50clusterCentroids.to_csv("year2002_50clusterCentroids.csv")

### 100 clusters

In [89]:
clusters2002_100 = km_100.fit_predict(year2002)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 56502, cost: 2639247.0
Run 1, iteration: 2/100, moves: 5844, cost: 2639246.0
Run 1, iteration: 3/100, moves: 0, cost: 2639246.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 77935, cost: 2655672.0
Run 2, iteration: 2/100, moves: 9571, cost: 2655668.0
Run 2, iteration: 3/100, moves: 0, cost: 2655668.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 66648, cost: 2642302.0
Run 3, iteration: 2/100, moves: 12919, cost: 2637243.0
Run 3, iteration: 3/100, moves: 7591, cost: 2636693.0
Run 3, iteration: 4/100, moves: 155, cost: 2636693.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 79453, cost: 2669263.0
Run 4, iteration: 2/100, moves: 7397, cost: 2669262.0
Run 4, iteration: 3/100, moves: 0,

In [90]:
print(clusters2002_100)
print(len(clusters2002_100))
print(len(year2002))

[50 16 14 ..., 16 16 16]
490879
490879


In [92]:
km_100.labels_

array([50, 16, 14, ..., 16, 16, 16], dtype=uint8)

In [93]:
km_100.cluster_centroids_

array([['1882', '1320', 'CRIMINAL DAMAGE', ..., '5', '6', '1'],
       ['372', '0820', 'THEFT', ..., '12', '25', '10'],
       ['3499', '0820', 'THEFT', ..., '7', '23', '7'],
       ..., 
       ['3791', '041A', 'BATTERY', ..., '4', '17', '20'],
       ['3757', '0560', 'ASSAULT', ..., '5', '15', '9'],
       ['658', '0560', 'ASSAULT', ..., '8', '24', '11']], 
      dtype='<U32')

In [94]:
year2002_100clusters = pd.DataFrame(clusters2002_100)

In [95]:
year2002_100clusters.columns = ['clusters_100']

In [96]:
year2002_100clusters = year2002_100clusters.reset_index()

In [97]:
year2002_100clusters['clusters_100'].value_counts()

8     23471
1     16886
0     15024
79    13773
43    13382
3     13276
12    13166
69    12479
18    12130
19    11785
14    11217
20    10617
4     10577
44    10531
31     9567
11     9407
7      8519
36     8348
16     7652
25     7576
15     7212
2      6779
10     6745
22     6642
58     6627
6      6580
33     6405
32     6174
82     5833
78     5794
      ...  
45     2238
57     2198
86     2188
92     2169
24     2116
63     2083
88     2001
93     1948
76     1899
75     1869
60     1869
48     1731
34     1668
94     1655
91     1644
66     1503
99     1455
38     1417
47     1406
84     1376
62     1313
54     1309
89     1280
77     1239
85     1226
68     1150
55     1112
87     1078
83     1068
72     1051
Name: clusters_100, dtype: int64

In [98]:
year2002New100 = pd.concat([year2002, year2002_100clusters], axis = 1)

In [100]:
year2002New100.to_csv("year2002_100clusters.csv")

In [101]:
year2002_100clusterCentroids = pd.DataFrame(km_100.cluster_centroids_)

In [102]:
year2002_100clusterCentroids.columns = year2002.columns

In [104]:
year2002_100clusterCentroids.to_csv("year2002_100clusterCentroids.csv")

## YEAR2003

In [105]:
year2003 = filterYear(2003)

In [106]:
year2003 = year2003.reset_index()

In [83]:
year2003.to_csv("year2003.csv")

### 5 clusters

In [107]:
clusters2003_5 = km_5.fit_predict(year2003)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 63295, cost: 3521574.0
Run 1, iteration: 2/100, moves: 11576, cost: 3521574.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 194880, cost: 3442971.0
Run 2, iteration: 2/100, moves: 34672, cost: 3425860.0
Run 2, iteration: 3/100, moves: 5910, cost: 3425860.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 72410, cost: 3448887.0
Run 3, iteration: 2/100, moves: 0, cost: 3448887.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 40850, cost: 3500512.0
Run 4, iteration: 2/100, moves: 5488, cost: 3500512.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 85399, cost: 3492636.0
Run 5, iteration: 2/100, moves: 10022, cost: 3492636.0


In [108]:
print(clusters2003_5)
print(len(clusters2003_5))
print(len(year2003))

[2 0 0 ..., 0 0 0]
475913
475913


In [109]:
km_5.labels_

array([2, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [110]:
km_5.cluster_centroids_

array([['4', '0820', 'THEFT', '$500 AND UNDER', 'STREET', 'False', 'False',
        '8.0', '2003.0', '7', '2', '13'],
       ['22', '0910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET',
        'False', 'False', '4.0', '2003.0', '8', '8', '14'],
       ['1', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE',
        'False', 'False', '7.0', '2003.0', '12', '14', '19'],
       ['244', '0486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE',
        'False', 'True', '11.0', '2003.0', '7', '20', '15'],
       ['2890', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS',
        'STREET', 'True', 'False', '11.0', '2003.0', '10', '6', '22']], 
      dtype='<U32')

In [111]:
year2003_5clusters = pd.DataFrame(clusters2003_5)

In [112]:
year2003_5clusters

,0
0,2
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,1


In [113]:
year2003_5clusters.columns = ['clusters_5']

In [114]:
year2003_5clusters['clusters_5'].value_counts()

0    195144
4     87476
2     75136
1     61027
3     57130
Name: clusters_5, dtype: int64

In [115]:
year2003New5 = pd.concat([year2003, year2003_5clusters], axis = 1)

In [116]:
year2003New5

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour,clusters_5
0,1,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,True,9.0,2003.0,3,1,0,2
1,4,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,22.0,2003.0,5,1,1,0
2,7,5007,OTHER OFFENSE,OTHER WEAPONS VIOLATION,RESIDENCE PORCH/HALLWAY,False,False,22.0,2003.0,3,15,0,0
3,9,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,11.0,2003.0,1,1,0,0
4,10,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,22.0,2003.0,6,5,0,0
5,12,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,8.0,2003.0,1,1,0,0
6,14,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,9.0,2003.0,11,1,8,0
7,15,0840,THEFT,FINANCIAL ID THEFT: OVER $300,APARTMENT,False,False,8.0,2003.0,11,15,9,0
8,16,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,14.0,2003.0,1,2,8,0
9,22,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,GAS STATION,False,False,7.0,2003.0,10,31,14,1


In [117]:
year2003New5.to_csv("year2003_5clusters.csv")

In [118]:
year2003_5clusterCentroids = pd.DataFrame(km_5.cluster_centroids_)

In [120]:
year2003_5clusterCentroids.columns = year2003.columns

In [122]:
year2003_5clusterCentroids.to_csv("year2003_5clusterCentroids.csv")

### 10 clusters

In [123]:
cluster2003_10 = km_10.fit_predict(year2003)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 122120, cost: 3186105.0
Run 1, iteration: 2/100, moves: 17880, cost: 3186104.0
Run 1, iteration: 3/100, moves: 0, cost: 3186104.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 70850, cost: 3243281.0
Run 2, iteration: 2/100, moves: 1086, cost: 3243281.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 133831, cost: 3180287.0
Run 3, iteration: 2/100, moves: 34027, cost: 3170834.0
Run 3, iteration: 3/100, moves: 11470, cost: 3170834.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 115730, cost: 3254868.0
Run 4, iteration: 2/100, moves: 22308, cost: 3254867.0
Run 4, iteration: 3/100, moves: 0, cost: 3254867.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
R

In [124]:
print(cluster2003_10)
print(len(cluster2003_10))
print(len(year2003))

[3 4 0 ..., 1 5 7]
475913
475913


In [125]:
km_10.labels_

array([3, 4, 0, ..., 1, 5, 7], dtype=uint8)

In [126]:
km_10.cluster_centroids_

array([['7', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE',
        'False', 'False', '16.0', '2003.0', '1', '16', '23'],
       ['31', '0820', 'THEFT', '$500 AND UNDER', 'STREET', 'False',
        'False', '1.0', '2003.0', '10', '19', '9'],
       ['798', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS',
        'STREET', 'True', 'False', '11.0', '2003.0', '11', '5', '20'],
       ['1', '0486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE',
        'False', 'True', '7.0', '2003.0', '7', '20', '18'],
       ['4', '0890', 'THEFT', 'FROM BUILDING', 'RESIDENCE', 'False',
        'False', '12.0', '2003.0', '8', '18', '12'],
       ['22', '0910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET',
        'False', 'False', '6.0', '2003.0', '11', '15', '21'],
       ['425', '0460', 'BATTERY', 'SIMPLE', 'SIDEWALK', 'False', 'False',
        '7.0', '2003.0', '11', '7', '14'],
       ['9', '0840', 'THEFT', 'FINANCIAL ID THEFT: OVER $300', 'RESIDENCE',
        'False', 'False', '8.0', 

In [127]:
year2003_10clusters = pd.DataFrame(cluster2003_10)

In [128]:
year2003_10clusters.columns = ['clusters_10']

In [129]:
year2003_10clusters = year2003_10clusters.reset_index()

In [130]:
year2003_10clusters['clusters_10'].value_counts()

2    88373
0    84202
1    72239
3    52493
6    38863
4    36387
5    33044
9    26620
8    25415
7    18277
Name: clusters_10, dtype: int64

In [131]:
year2003New10 = pd.concat([year2003, year2003_10clusters], axis = 1)

In [133]:
year2003New10.to_csv("year2003_10clusters.csv")

In [134]:
year2003_10clusterCentroids = pd.DataFrame(km_10.cluster_centroids_)

In [136]:
year2003_10clusterCentroids.columns = year2003.columns

In [138]:
year2003_10clusterCentroids.to_csv("year2003_10clusterCentroids.csv")

### 25 clusters

In [157]:
cluster2003_25 = km_25.fit_predict(year2003)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 98868, cost: 2868637.0
Run 1, iteration: 2/100, moves: 3586, cost: 2867861.0
Run 1, iteration: 3/100, moves: 235, cost: 2867861.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 124677, cost: 2920343.0
Run 2, iteration: 2/100, moves: 2332, cost: 2920343.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 79254, cost: 2937670.0
Run 3, iteration: 2/100, moves: 3774, cost: 2937670.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 119218, cost: 2921620.0
Run 4, iteration: 2/100, moves: 4981, cost: 2921205.0
Run 4, iteration: 3/100, moves: 408, cost: 2921205.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 86910, cost: 2906051.0
R

In [158]:
print(cluster2003_25)
print(len(cluster2003_25))
print(len(year2003))

[ 5 11 10 ..., 18  8  2]
475913
475913


In [159]:
km_25.labels_

array([ 5, 11, 10, ..., 18,  8,  2], dtype=uint8)

In [160]:
km_25.cluster_centroids_

array([['39', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', 'False',
        'False', '19.0', '2003.0', '7', '2', '22'],
       ['1121', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS',
        'STREET', 'True', 'False', '25.0', '2003.0', '11', '22', '15'],
       ['529', '0820', 'THEFT', '$500 AND UNDER', 'STREET', 'False',
        'False', '9.0', '2003.0', '9', '28', '19'],
       ['40', '2820', 'OTHER OFFENSE', 'TELEPHONE THREAT', 'RESIDENCE',
        'False', 'False', '5.0', '2003.0', '8', '24', '18'],
       ['145', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', 'False',
        'True', '10.0', '2003.0', '10', '11', '0'],
       ['1', '2825', 'OTHER OFFENSE', 'HARASSMENT BY TELEPHONE',
        'RESIDENCE', 'False', 'False', '20.0', '2003.0', '7', '25', '1'],
       ['125', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE',
        'True', 'False', '8.0', '2003.0', '1', '25', '0'],
       ['101', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS',
        'STREET'

In [161]:
year2003_25clusters = pd.DataFrame(cluster2003_25)

In [162]:
year2003_25clusters.columns = ['clusters_25']

In [163]:
year2003_25clusters = year2003_25clusters.reset_index()

In [164]:
year2003_25clusters['clusters_25'].value_counts()

2     43792
0     42351
1     39312
19    37972
14    27303
8     24220
11    24049
15    23123
10    21824
22    20744
20    20149
3     19777
7     16979
9     16375
12    14819
6     14140
17    13045
5     12791
23     8563
21     7702
16     6709
24     6401
18     6081
4      3963
13     3729
Name: clusters_25, dtype: int64

In [165]:
year2003New25 = pd.concat([year2003, year2003_25clusters], axis = 1)

In [167]:
year2003New25.to_csv("year2003_25clusters.csv")

In [168]:
year2003_25clusterCentroids = pd.DataFrame(km_25.cluster_centroids_)

In [169]:
year2003_25clusterCentroids.columns = year2003.columns

In [170]:
year2003_25clusterCentroids.to_csv("year2003_25clusterCentroids.csv")

### 50 clusters

In [139]:
cluster2003_50 = km_50.fit_predict(year2003)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 86120, cost: 2745454.0
Run 1, iteration: 2/100, moves: 3276, cost: 2745451.0
Run 1, iteration: 3/100, moves: 0, cost: 2745451.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 68913, cost: 2722061.0
Run 2, iteration: 2/100, moves: 6797, cost: 2719817.0
Run 2, iteration: 3/100, moves: 1212, cost: 2719814.0
Run 2, iteration: 4/100, moves: 0, cost: 2719814.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 112608, cost: 2706292.0
Run 3, iteration: 2/100, moves: 24662, cost: 2704885.0
Run 3, iteration: 3/100, moves: 3637, cost: 2701641.0
Run 3, iteration: 4/100, moves: 214, cost: 2701641.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 135353, cost: 2716767.0
Run 4, iteration: 2/100, moves: 

In [140]:
print(cluster2003_50)
print(len(cluster2003_50))
print(len(year2003))

[37  5 37 ...,  2  1  3]
475913
475913


In [141]:
km_50.labels_

array([37,  5, 37, ...,  2,  1,  3], dtype=uint8)

In [142]:
km_50.cluster_centroids_

array([['683', '2095', 'NARCOTICS', 'ATTEMPT POSSESSION NARCOTICS',
        'STREET', 'True', 'False', '11.0', '2003.0', '7', '13', '1'],
       ['165', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', 'False',
        'False', '11.0', '2003.0', '6', '14', '12'],
       ['14', '0890', 'THEFT', 'FROM BUILDING', 'RESIDENCE', 'False',
        'False', '25.0', '2003.0', '11', '27', '9'],
       ['297', '0890', 'THEFT', 'FROM BUILDING', 'APARTMENT', 'False',
        'False', '12.0', '2003.0', '10', '23', '22'],
       ['39', '0820', 'THEFT', '$500 AND UNDER', 'STREET', 'False',
        'False', '4.0', '2003.0', '8', '12', '18'],
       ['4', '0841', 'THEFT', 'FINANCIAL ID THEFT:$300 &UNDER',
        'RESIDENCE', 'False', 'False', '22.0', '2003.0', '10', '27', '10'],
       ['100', '0560', 'ASSAULT', 'SIMPLE', 'STREET', 'False', 'False',
        '9.0', '2003.0', '9', '6', '22'],
       ['613', '2027', 'NARCOTICS', 'POSS: CRACK', 'SIDEWALK', 'True',
        'False', '11.0', '2003.0', '11', 

In [143]:
year2003_50clusters = pd.DataFrame(cluster2003_50)

In [145]:
year2003_50clusters.columns = ['clusters_50']

In [146]:
year2003_50clusters = year2003_50clusters.reset_index()

In [147]:
year2003_50clusters['clusters_50'].value_counts()

4     29190
13    25527
0     22390
2     22313
10    21522
22    21401
16    21359
7     20659
6     19350
1     17671
9     15760
21    14757
12    13581
17    12830
34    11278
37    11011
8     10498
14     9813
47     9292
40     9271
20     8827
3      8436
18     8388
19     7584
35     7431
15     7379
11     6816
23     6759
31     5543
5      5509
26     5020
41     4592
43     4447
49     4424
36     4380
45     4180
38     4134
33     3827
30     3622
25     3113
48     3010
29     2967
28     2905
46     2627
24     2570
44     2374
42     1958
32     1624
39     1247
27      747
Name: clusters_50, dtype: int64

In [148]:
year2003New50 = pd.concat([year2003, year2003_50clusters], axis = 1)

In [150]:
year2003New50.to_csv("year2003_50clusters.csv")

In [151]:
year2003_50clusterCentroids = pd.DataFrame(km_50.cluster_centroids_)

In [153]:
year2003_50clusterCentroids.columns = year2003.columns

In [155]:
year2003_50clusterCentroids.to_csv("year2003_50clusterCentroids.csv")

### YEAR2004

In [20]:
year2004 = filterYear(2004)

In [21]:
year2004 = year2004.reset_index()

In [92]:
year2004.to_csv("year2004.csv")

### 5 clusters

In [173]:
clusters2004_5 = km_5.fit_predict(year2004)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 86470, cost: 2793865.0
Run 1, iteration: 2/100, moves: 27598, cost: 2789938.0
Run 1, iteration: 3/100, moves: 11401, cost: 2784753.0
Run 1, iteration: 4/100, moves: 19, cost: 2784753.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 72387, cost: 2847264.0
Run 2, iteration: 2/100, moves: 5660, cost: 2847264.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 87822, cost: 2812849.0
Run 3, iteration: 2/100, moves: 25504, cost: 2811942.0
Run 3, iteration: 3/100, moves: 15390, cost: 2802005.0
Run 3, iteration: 4/100, moves: 908, cost: 2802005.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 85544, cost: 2777485.0
Run 4, iteration: 2/100, moves: 3797, cost: 2777485.0
Init: initializing centroid

In [174]:
print(clusters2004_5)
print(len(clusters2004_5))
print(len(year2004))

[0 4 4 ..., 0 2 2]
388205
388205


In [175]:
km_5.labels_

array([0, 4, 4, ..., 0, 2, 2], dtype=uint8)

In [176]:
km_5.cluster_centroids_

array([['0', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'APARTMENT',
        'False', 'False', '8.0', '2004.0', '1', '10', '13'],
       ['41', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS',
        'STREET', 'True', 'False', '11.0', '2004.0', '9', '8', '11'],
       ['8', '0820', 'THEFT', '$500 AND UNDER', 'STREET', 'False', 'False',
        '12.0', '2004.0', '7', '7', '21'],
       ['45', '0486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE',
        'False', 'True', '8.0', '2004.0', '10', '6', '12'],
       ['2', '0460', 'BATTERY', 'SIMPLE', 'RESIDENCE', 'False', 'False',
        '25.0', '2004.0', '8', '14', '20']], 
      dtype='<U32')

In [177]:
year2004_5clusters = pd.DataFrame(clusters2004_5)

In [180]:
year2004_5clusters.columns = ['clusters_5']

In [184]:
year2004_5clusters = year2004_5clusters.reset_index()

In [185]:
year2004_5clusters['clusters_5'].value_counts()

0    98404
2    97034
1    90296
4    56645
3    45826
Name: clusters_5, dtype: int64

In [186]:
year2004New5 = pd.concat([year2004, year2004_5clusters], axis = 1)

In [187]:
year2004New5

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour,index,clusters_5
0,0,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,4.0,2004.0,1,1,0,0,0
1,2,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,False,14.0,2004.0,6,20,11,1,4
2,3,0840,THEFT,FINANCIAL ID THEFT: OVER $300,OTHER,False,False,25.0,2004.0,12,30,20,2,4
3,5,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,APARTMENT,False,False,10.0,2004.0,8,1,0,3,0
4,8,0890,THEFT,FROM BUILDING,RESIDENCE,False,False,18.0,2004.0,9,16,10,4,2
5,11,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,20.0,2004.0,11,1,0,5,2
6,13,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,4.0,2004.0,11,1,0,6,2
7,21,0840,THEFT,FINANCIAL ID THEFT: OVER $300,APARTMENT,False,False,1.0,2004.0,1,1,0,7,0
8,27,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,RESIDENCE,False,False,3.0,2004.0,12,13,13,8,0
9,28,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,25.0,2004.0,10,6,9,9,3


In [188]:
year2004New5.to_csv("year2004_5clusters.csv")

2008.0    852053
2006.0    794684
2009.0    783900
2010.0    700691
2007.0    621848
2001.0    568517
2002.0    490879
2003.0    475913
2005.0    455811
2004.0    388205
2011.0    352066
2012.0    335670
2013.0    306703
2014.0    274527
2016.0    265462
2015.0    262995
2017.0     11357
Name: Year, dtype: int64

In [189]:
year2004_5clusterCentroids = pd.DataFrame(km_5.cluster_centroids_)

In [190]:
year2004_5clusterCentroids.columns = year2004.columns

In [191]:
year2004_5clusterCentroids

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour
0,0,1310,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,False,False,8.0,2004.0,1,10,13
1,41,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,STREET,True,False,11.0,2004.0,9,8,11
2,8,0820,THEFT,$500 AND UNDER,STREET,False,False,12.0,2004.0,7,7,21
3,45,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,8.0,2004.0,10,6,12
4,2,0460,BATTERY,SIMPLE,RESIDENCE,False,False,25.0,2004.0,8,14,20


In [192]:
year2004_5clusterCentroids.to_csv("year2004_5clusterCentroids.csv")

### 10 clusters

In [22]:
clusters2004_10 = km_10.fit_predict(year2004)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 82570, cost: 2604373.0
Run 1, iteration: 2/100, moves: 6806, cost: 2604373.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 97417, cost: 2665040.0
Run 2, iteration: 2/100, moves: 9576, cost: 2665040.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 110601, cost: 2584699.0
Run 3, iteration: 2/100, moves: 26184, cost: 2581452.0
Run 3, iteration: 3/100, moves: 1164, cost: 2581452.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 72660, cost: 2625130.0
Run 4, iteration: 2/100, moves: 19475, cost: 2620250.0
Run 4, iteration: 3/100, moves: 1185, cost: 2620250.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 101752, cost: 2604921

In [23]:
print(clusters2004_10)
print(len(clusters2004_10))
print(len(year2004))

[1 1 1 ..., 2 9 8]
388205
388205


In [24]:
km_10.labels_

array([1, 1, 1, ..., 2, 9, 8], dtype=uint8)

In [25]:
km_10.cluster_centroids_

array([['74', '0910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET',
        'False', 'False', '15.0', '2004.0', '5', '19', '21'],
       ['0', '0486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE',
        'False', 'False', '25.0', '2004.0', '6', '10', '16'],
       ['5', '0810', 'THEFT', 'OVER $500', 'STREET', 'False', 'False',
        '6.0', '2004.0', '8', '12', '12'],
       ['180', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', 'False',
        'False', '5.0', '2004.0', '1', '6', '22'],
       ['67', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS',
        'SIDEWALK', 'True', 'False', '11.0', '2004.0', '7', '8', '19'],
       ['146', '0820', 'THEFT', '$500 AND UNDER', 'STREET', 'False',
        'False', '12.0', '2004.0', '6', '14', '22'],
       ['46', '0460', 'BATTERY', 'SIMPLE', 'STREET', 'True', 'False',
        '6.0', '2004.0', '12', '18', '22'],
       ['11', '0820', 'THEFT', '$500 AND UNDER', 'SIDEWALK', 'False',
        'False', '19.0', '2004.0', '11', '19', '22

In [26]:
year2004_10clusters = pd.DataFrame(clusters2004_10)

In [27]:
year2004_10clusters.columns = ['clusters_10']

In [29]:
year2004_10clusters = year2004_10clusters.reset_index()

In [31]:
year2004_10clusters['clusters_10'].value_counts()

1    71487
4    53868
0    46937
2    46418
6    37461
5    36071
3    32597
9    27655
8    23976
7    11735
Name: clusters_10, dtype: int64

In [32]:
year2004New10 = pd.concat([year2004, year2004_10clusters], axis = 1)

In [34]:
year2004New10.to_csv("year2004_10clusters.csv")

In [35]:
year2004_10clusterCentroids = pd.DataFrame(km_10.cluster_centroids_)

In [36]:
year2004_10clusterCentroids.columns = year2004.columns

In [38]:
year2004_10clusterCentroids.to_csv("year2004_10clusterCentroids.csv")

### 25 clusters

In [39]:
clusters2004_25 = km_25.fit_predict(year2004)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 80155, cost: 2362056.0
Run 1, iteration: 2/100, moves: 30813, cost: 2357953.0
Run 1, iteration: 3/100, moves: 3302, cost: 2357952.0
Run 1, iteration: 4/100, moves: 0, cost: 2357952.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 87351, cost: 2385822.0
Run 2, iteration: 2/100, moves: 27809, cost: 2373806.0
Run 2, iteration: 3/100, moves: 4599, cost: 2373805.0
Run 2, iteration: 4/100, moves: 0, cost: 2373805.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 71509, cost: 2344853.0
Run 3, iteration: 2/100, moves: 6251, cost: 2344853.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 83186, cost: 2369381.0
Run 4, iteration: 2/100, moves: 10785, cost: 2369380.0
Run 4, iteration: 3/100, moves:

In [40]:
print(clusters2004_25)
print(len(clusters2004_25))
print(len(year2004))

[ 1 11  2 ...,  5  5  5]
388205
388205


In [41]:
km_25.labels_

array([ 1, 11,  2, ...,  5,  5,  5], dtype=uint8)

In [42]:
km_25.cluster_centroids_

array([['46', '0460', 'BATTERY', 'SIMPLE', 'STREET', 'False', 'False',
        '17.0', '2004.0', '7', '27', '14'],
       ['0', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', 'False',
        'False', '14.0', '2004.0', '1', '4', '0'],
       ['3', '0610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE', 'False',
        'False', '6.0', '2004.0', '12', '5', '14'],
       ['45', '0820', 'THEFT', '$500 AND UNDER', 'STREET', 'False',
        'False', '8.0', '2004.0', '6', '10', '13'],
       ['158', '0460', 'BATTERY', 'SIMPLE', 'STREET', 'False', 'False',
        '5.0', '2004.0', '8', '29', '16'],
       ['11', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE',
        'False', 'False', '19.0', '2004.0', '10', '1', '0'],
       ['257', '0460', 'BATTERY', 'SIMPLE', 'RESIDENCE', 'False', 'False',
        '8.0', '2004.0', '5', '19', '23'],
       ['179', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS',
        'STREET', 'True', 'False', '25.0', '2004.0', '10', '19', '22'],
       ['157'

In [43]:
year2004_25clusters = pd.DataFrame(clusters2004_25)

In [44]:
year2004_25clusters.columns = ['clusters_25']

In [45]:
year2004_25clusters = year2004_25clusters.reset_index()

In [47]:
year2004_25clusters['clusters_25'].value_counts()

3     39734
0     36188
1     30304
7     27878
2     27630
5     27517
9     21091
11    18819
23    16861
14    15830
12    14419
4     14348
10    13165
6     12099
17    11944
24    10400
19     9679
18     8293
13     6518
21     5744
15     5208
16     4982
8      4040
22     3654
20     1860
Name: clusters_25, dtype: int64

In [48]:
year2004New25 = pd.concat([year2004, year2004_25clusters], axis = 1)

In [50]:
year2004New25.to_csv("year2004_25clusters.csv")

In [51]:
year2004_25clusterCentroids = pd.DataFrame(km_25.cluster_centroids_)

In [52]:
year2004_25clusterCentroids.columns = year2004.columns

In [54]:
year2004_25clusterCentroids.to_csv("year2004_25clusterCentroids.csv")

### 50 clusters

In [55]:
clusters2004_50 = km_50.fit_predict(year2004)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 82994, cost: 2188901.0
Run 1, iteration: 2/100, moves: 3869, cost: 2188900.0
Run 1, iteration: 3/100, moves: 0, cost: 2188900.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 60143, cost: 2220800.0
Run 2, iteration: 2/100, moves: 7189, cost: 2220204.0
Run 2, iteration: 3/100, moves: 503, cost: 2220204.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 59988, cost: 2230496.0
Run 3, iteration: 2/100, moves: 6352, cost: 2230491.0
Run 3, iteration: 3/100, moves: 0, cost: 2230491.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 91511, cost: 2197241.0
Run 4, iteration: 2/100, moves: 5763, cost: 2197240.0
Run 4, iteration: 3/100, moves: 0, cost: 2197240.0
Init: initializing centroids
Init: ini

In [56]:
print(clusters2004_50)
print(len(clusters2004_50))
print(len(year2004))

[10  6 17 ..., 10 10 10]
388205
388205


In [67]:
km_50.labels_

array([10,  6, 17, ..., 10, 10, 10], dtype=uint8)

In [68]:
km_50.cluster_centroids_

array([['86', '0320', 'ROBBERY', 'STRONGARM - NO WEAPON', 'STREET',
        'False', 'False', '9.0', '2004.0', '9', '5', '3'],
       ['67', '0560', 'ASSAULT', 'SIMPLE', 'APARTMENT', 'True', 'False',
        '15.0', '2004.0', '11', '9', '12'],
       ['48', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS',
        'STREET', 'True', 'False', '12.0', '2004.0', '7', '27', '22'],
       ['178', '031A', 'ROBBERY', 'ARMED: HANDGUN', 'STREET', 'True',
        'False', '6.0', '2004.0', '12', '2', '21'],
       ['144', '0486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'APARTMENT',
        'True', 'False', '24.0', '2004.0', '1', '19', '1'],
       ['160', '0460', 'BATTERY', 'SIMPLE', 'STREET', 'True', 'False',
        '8.0', '2004.0', '1', '9', '11'],
       ['673', '0560', 'ASSAULT', 'SIMPLE', 'RESIDENCE', 'False', 'True',
        '12.0', '2004.0', '2', '16', '11'],
       ['27', '0610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE', 'False',
        'False', '7.0', '2004.0', '1', '2', '18'],
      

In [57]:
year2004_50clusters = pd.DataFrame(clusters2004_50)

In [59]:
year2004_50clusters.columns = ['clusters_50']

In [60]:
year2004_50clusters = year2004_50clusters.reset_index()

In [62]:
year2004_50clusters['clusters_50'].value_counts()

7     20900
2     19533
15    16499
23    16052
18    14620
31    14470
10    14234
5     13873
12    12803
9     12225
32    12142
20    11791
17    10049
11     9811
0      9775
36     9261
4      8986
1      8874
45     8583
48     8145
25     7974
38     7790
49     7692
13     7575
21     7524
28     7397
14     6458
3      5975
6      5742
34     5645
40     5527
30     5397
42     4642
19     4588
16     4534
46     4410
8      4117
33     3763
35     3754
39     3335
44     3046
47     2997
22     2681
29     2508
37     2505
24     2034
43     1903
26     1521
27     1435
41     1110
Name: clusters_50, dtype: int64

In [63]:
year2004New50 = pd.concat([year2004, year2004_50clusters], axis = 1)

In [111]:
year2004New50

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour,index,clusters_50
0,0,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,4.0,2004.0,1,1,0,0,10
1,2,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,False,14.0,2004.0,6,20,11,1,6
2,3,0840,THEFT,FINANCIAL ID THEFT: OVER $300,OTHER,False,False,25.0,2004.0,12,30,20,2,17
3,5,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,APARTMENT,False,False,10.0,2004.0,8,1,0,3,10
4,8,0890,THEFT,FROM BUILDING,RESIDENCE,False,False,18.0,2004.0,9,16,10,4,23
5,11,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,20.0,2004.0,11,1,0,5,10
6,13,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,4.0,2004.0,11,1,0,6,10
7,21,0840,THEFT,FINANCIAL ID THEFT: OVER $300,APARTMENT,False,False,1.0,2004.0,1,1,0,7,10
8,27,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,RESIDENCE,False,False,3.0,2004.0,12,13,13,8,7
9,28,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,25.0,2004.0,10,6,9,9,44


In [65]:
year2004New50.to_csv("year2004_50clusters.csv")

In [66]:
year2004_50clusterCentroids = pd.DataFrame(km_50.cluster_centroids_)

In [69]:
year2004_50clusterCentroids.columns = year2004.columns

In [71]:
year2004_50clusterCentroids.to_csv("year2004_50clusterCentroids.csv")

## YEAR2005

In [72]:
year2005 = filterYear(2005)

In [73]:
year2005 = year2005.reset_index()

In [75]:
year2005.to_csv("year2005.csv")

### 5 clusters

In [76]:
clusters2005_5 = km_5.fit_predict(year2005)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 103635, cost: 3260519.0
Run 1, iteration: 2/100, moves: 4754, cost: 3260519.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 89160, cost: 3288982.0
Run 2, iteration: 2/100, moves: 1347, cost: 3288982.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 137296, cost: 3411278.0
Run 3, iteration: 2/100, moves: 0, cost: 3411278.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 101385, cost: 3359814.0
Run 4, iteration: 2/100, moves: 7238, cost: 3359814.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 35018, cost: 3342507.0
Run 5, iteration: 2/100, moves: 28694, cost: 3322778.0
Run 5, iteration: 3/100, moves: 18693, cost: 3321857.0

In [77]:
print(clusters2005_5)
print(len(clusters2005_5))
print(len(year2005))

[1 3 3 ..., 1 3 3]
455811
455811


In [78]:
km_5.labels_

array([1, 3, 3, ..., 1, 3, 3], dtype=uint8)

In [79]:
km_5.cluster_centroids_

array([['1924301', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET',
        'False', 'False', '3.0', '2005.0', '12', '28', '23'],
       ['1923654', '0460', 'BATTERY', 'SIMPLE', 'STREET', 'False', 'False',
        '8.0', '2005.0', '9', '26', '18'],
       ['1923976', '0486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE',
        'RESIDENCE', 'False', 'True', '7.0', '2005.0', '5', '1', '13'],
       ['1923843', '0820', 'THEFT', '$500 AND UNDER', 'RESIDENCE', 'False',
        'False', '25.0', '2005.0', '3', '14', '12'],
       ['1923886', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS',
        'STREET', 'True', 'False', '11.0', '2005.0', '7', '8', '20']], 
      dtype='<U32')

In [117]:
year2005_5clusters = pd.DataFrame(clusters2005_5)

In [118]:
year2005_5clusters.columns = ['clusters_5']

In [84]:
year2005_5clusters['clusters_5'].value_counts()

0    131906
3     92753
1     92200
4     83774
2     55178
Name: clusters_5, dtype: int64

In [119]:
year2005New5 = pd.concat([year2005, year2005_5clusters], axis = 1)

In [120]:
year2005New5

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour,clusters_5
0,1923654,0560,ASSAULT,SIMPLE,"SCHOOL, PUBLIC, BUILDING",False,False,9.0,2005.0,10,21,11,1
1,1923843,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,15.0,2005.0,7,14,0,3
2,1923876,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,2.0,2005.0,12,26,15,3
3,1923886,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,STREET,True,False,6.0,2005.0,3,24,15,4
4,1923887,2027,NARCOTICS,POSS: CRACK,STREET,True,False,6.0,2005.0,3,30,0,4
5,1923964,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,19.0,2005.0,2,5,12,3
6,1923976,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,True,True,14.0,2005.0,12,25,2,2
7,1923992,0810,THEFT,OVER $500,VEHICLE NON-COMMERCIAL,False,False,12.0,2005.0,10,2,1,3
8,1924059,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,False,19.0,2005.0,11,7,12,3
9,1924295,1195,DECEPTIVE PRACTICE,FINAN EXPLOIT-ELDERLY/DISABLED,RESIDENCE,False,False,22.0,2005.0,11,1,0,2


In [127]:
year2005New5.to_csv("year2005_5clusters.csv")

In [87]:
year2005_5clusterCentroids = pd.DataFrame(km_5.cluster_centroids_)

In [89]:
year2005_5clusterCentroids.columns = year2005.columns

In [90]:
year2005_5clusterCentroids.to_csv("year2005_5clusterCentroids.csv")

### 10 clusters

In [91]:
clusters2005_10 = km_10.fit_predict(year2005)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 119075, cost: 3124451.0
Run 1, iteration: 2/100, moves: 36573, cost: 3124450.0
Run 1, iteration: 3/100, moves: 0, cost: 3124450.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 96453, cost: 3094865.0
Run 2, iteration: 2/100, moves: 7009, cost: 3094865.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 85103, cost: 3057842.0
Run 3, iteration: 2/100, moves: 17314, cost: 3057842.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 88661, cost: 3178156.0
Run 4, iteration: 2/100, moves: 12188, cost: 3174454.0
Run 4, iteration: 3/100, moves: 353, cost: 3174454.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 129293, cost: 3081913.0


In [92]:
print(clusters2005_10)
print(len(clusters2005_10))
print(len(year2005))

[0 0 0 ..., 1 5 0]
455811
455811


In [93]:
km_10.labels_

array([0, 0, 0, ..., 1, 5, 0], dtype=uint8)

In [95]:
km_10.cluster_centroids_

array([['1923654', '0820', 'THEFT', '$500 AND UNDER', 'STREET', 'False',
        'False', '9.0', '2005.0', '11', '23', '0'],
       ['1924734', '0460', 'BATTERY', 'SIMPLE', 'SIDEWALK', 'False',
        'False', '5.0', '2005.0', '9', '15', '10'],
       ['1923886', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS',
        'STREET', 'True', 'False', '15.0', '2005.0', '4', '28', '14'],
       ['1923976', '0486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE',
        'RESIDENCE', 'False', 'True', '7.0', '2005.0', '7', '7', '21'],
       ['1924568', '0860', 'THEFT', 'RETAIL THEFT', 'GROCERY FOOD STORE',
        'True', 'False', '12.0', '2005.0', '9', '3', '13'],
       ['1923964', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET',
        'False', 'False', '10.0', '2005.0', '6', '1', '12'],
       ['1924396', '0890', 'THEFT', 'FROM BUILDING', 'OTHER', 'False',
        'False', '1.0', '2005.0', '9', '20', '0'],
       ['1924301', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'APARTMENT',
        'Fals

In [112]:
year2005_10clusters = pd.DataFrame(clusters2005_10)

In [113]:
year2005_10clusters.columns = ['clusters_10']

In [114]:
year2005_10clusters

,clusters_10
0,0
1,0
2,0
3,2
4,9
5,5
6,3
7,0
8,0
9,0


In [115]:
year2005New10 = pd.concat([year2005, year2005_10clusters], axis = 1)

In [121]:
year2005New10.to_csv("year2005_10clusters.csv")

In [122]:
year2005_10clusterCentroids = pd.DataFrame(km_10.cluster_centroids_)

In [124]:
year2005_10clusterCentroids.columns = year2005.columns

In [126]:
year2005_10clusterCentroids.to_csv("year2005_10clusterCentroids.csv")

### 25 clusters

In [128]:
clusters2005_25 = km_25.fit_predict(year2005)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 91158, cost: 2809402.0
Run 1, iteration: 2/100, moves: 5227, cost: 2809402.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 98114, cost: 2862183.0
Run 2, iteration: 2/100, moves: 14428, cost: 2862182.0
Run 2, iteration: 3/100, moves: 0, cost: 2862182.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 110320, cost: 2731327.0
Run 3, iteration: 2/100, moves: 12278, cost: 2731327.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 117759, cost: 2780357.0
Run 4, iteration: 2/100, moves: 35964, cost: 2780355.0
Run 4, iteration: 3/100, moves: 0, cost: 2780355.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 105739, cost: 2772915.0
R

In [130]:
print(clusters2005_25)
print(len(clusters2005_25))
print(len(year2005))

[ 8 22 10 ...,  8 10  5]
455811
455811


In [131]:
km_25.labels_

array([ 8, 22, 10, ...,  8, 10,  5], dtype=uint8)

In [132]:
km_25.cluster_centroids_

array([['1924301', '0460', 'BATTERY', 'SIMPLE', 'SIDEWALK', 'False',
        'False', '25.0', '2005.0', '12', '27', '19'],
       ['1925730', '0610', 'BURGLARY', 'FORCIBLE ENTRY', 'APARTMENT',
        'False', 'False', '8.0', '2005.0', '10', '23', '13'],
       ['1924956', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE',
        'False', 'False', '5.0', '2005.0', '12', '27', '11'],
       ['1925151', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET',
        'False', 'False', '2.0', '2005.0', '7', '15', '18'],
       ['1923887', '2027', 'NARCOTICS', 'POSS: CRACK', 'STREET', 'True',
        'False', '11.0', '2005.0', '5', '13', '17'],
       ['1924059', '2820', 'OTHER OFFENSE', 'TELEPHONE THREAT',
        'RESIDENCE', 'False', 'False', '4.0', '2005.0', '4', '19', '12'],
       ['1923886', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS',
        'STREET', 'True', 'False', '4.0', '2005.0', '6', '14', '22'],
       ['1925889', '0860', 'THEFT', 'RETAIL THEFT', 'GROCERY FOOD STORE'

In [133]:
year2005_25clusters = pd.DataFrame(clusters2005_25)

In [134]:
year2005_25clusters.columns = ['clusters_25']

In [136]:
year2005_25clusters['clusters_25'].value_counts()

0     49152
11    33260
1     31083
3     30789
4     29666
22    27818
6     26596
2     25993
10    24027
8     22639
5     22572
16    20056
7     14051
12    12960
14    11438
13    11128
23     9988
18     9791
20     9350
19     9042
15     7018
9      6704
17     4990
24     3619
21     2081
Name: clusters_25, dtype: int64

In [137]:
year2005New25 = pd.concat([year2005, year2005_25clusters], axis = 1)

In [139]:
year2005New25.to_csv("year2005_25clusters.csv")

In [140]:
year2005_25clusterCentroids = pd.DataFrame(km_25.cluster_centroids_)

In [141]:
year2005_25clusterCentroids.columns = year2005.columns

In [143]:
year2005_25clusterCentroids.to_csv("year2005_25clusterCentroids.csv")

## YEAR 2006

In [174]:
year2006 = filterYear(2006)

In [175]:
year2006 = year2006.reset_index()

In [176]:
year2006.head()

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour
0,1923514,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,False,16.0,2006.0,4,2,13
1,1923515,2017,NARCOTICS,MANU/DELIVER:CRACK,SIDEWALK,True,False,3.0,2006.0,2,26,13
2,1923516,051A,ASSAULT,AGGRAVATED: HANDGUN,OTHER,False,False,3.0,2006.0,1,8,23
3,1923517,0460,BATTERY,SIMPLE,RESIDENCE,False,False,16.0,2006.0,4,5,18
4,1923518,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,ALLEY,True,False,8.0,2006.0,2,17,21


### 5 clusters

In [177]:
clusters2006_5 = km_5.fit_predict(year2006)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 123695, cost: 6040756.0
Run 1, iteration: 2/100, moves: 3932, cost: 6040756.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 133221, cost: 5798372.0
Run 2, iteration: 2/100, moves: 151694, cost: 5798372.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 163197, cost: 5730447.0
Run 3, iteration: 2/100, moves: 28328, cost: 5730447.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 228643, cost: 5725321.0
Run 4, iteration: 2/100, moves: 18732, cost: 5725321.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 241040, cost: 5746695.0
Run 5, iteration: 2/100, moves: 35663, cost: 5746695.0
Best run was number 4


In [187]:
year2006_5clusters = pd.DataFrame(clusters2006_5)

In [188]:
year2006_5clusters.columns = ['clusters_5']

In [190]:
year2006New5 = pd.concat([year2006, year2006_5clusters], axis = 1)

In [192]:
year2006New5.to_csv("year2006_5clusters.csv")

In [193]:
year2006_5clusterCentroids = pd.DataFrame(km_5.cluster_centroids_)

In [194]:
year2006_5clusterCentroids.columns = year2006.columns

In [196]:
year2006_5clusterCentroids.to_csv("year2006_5clusterCentroids.csv")

### 10 clusters

In [185]:
clusters2006_10 = km_10.fit_predict(year2006)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 99079, cost: 5501110.0
Run 1, iteration: 2/100, moves: 18840, cost: 5501110.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 221273, cost: 5334634.0
Run 2, iteration: 2/100, moves: 32010, cost: 5334633.0
Run 2, iteration: 3/100, moves: 0, cost: 5334633.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 236909, cost: 5407183.0
Run 3, iteration: 2/100, moves: 25869, cost: 5407183.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 215421, cost: 5353598.0
Run 4, iteration: 2/100, moves: 17587, cost: 5353597.0
Run 4, iteration: 3/100, moves: 0, cost: 5353597.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 222742, cost: 5487202.0

In [197]:
year2006_10clusters = pd.DataFrame(clusters2006_10)

In [199]:
year2006_10clusters.columns = ['clusters_10']

In [200]:
year2006New10 = pd.concat([year2006, year2006_10clusters], axis = 1)

In [202]:
year2006New10.to_csv("year2006_10clusters.csv")

In [203]:
year2006_10clusterCentroids = pd.DataFrame(km_10.cluster_centroids_)

In [204]:
year2006_10clusterCentroids.columns = year2006.columns

In [205]:
year2006_10clusterCentroids.to_csv("year2006_10clusterCentroids.csv")

### 25 clusters

In [186]:
clusters2006_25 = km_25.fit_predict(year2006)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 217626, cost: 4926179.0
Run 1, iteration: 2/100, moves: 38881, cost: 4922799.0
Run 1, iteration: 3/100, moves: 3487, cost: 4922799.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 172780, cost: 4828613.0
Run 2, iteration: 2/100, moves: 34712, cost: 4823253.0
Run 2, iteration: 3/100, moves: 2444, cost: 4823253.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 167432, cost: 4838897.0
Run 3, iteration: 2/100, moves: 17564, cost: 4835633.0
Run 3, iteration: 3/100, moves: 1636, cost: 4835633.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 166464, cost: 4928877.0
Run 4, iteration: 2/100, moves: 21590, cost: 4928877.0
Init: initializing centroids
Init: initializing clusters
Starting iteratio

In [206]:
year2006_25clusters = pd.DataFrame(clusters2006_25)

In [207]:
year2006_25clusters.columns = ['clusters_25']

In [209]:
year2006New25 = pd.concat([year2006, year2006_25clusters], axis = 1)

In [211]:
year2006New25.to_csv("year2006_25clusters.csv")

In [212]:
year2006_25clusterCentroids = pd.DataFrame(km_25.cluster_centroids_)

In [213]:
year2006_25clusterCentroids.columns = year2006.columns

In [215]:
year2006_25clusterCentroids.to_csv("year2006_25clusterCentroids.csv")

## YEAR 2007

In [252]:
year2007 = filterYear(2007)

In [253]:
year2007 = year2007.reset_index()

In [218]:
year2007.head()

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour
0,2180446,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,True,2.0,2007.0,1,3,10
1,2180876,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,12.0,2007.0,1,4,18
2,2180912,5002,OTHER OFFENSE,OTHER VEHICLE OFFENSE,STREET,True,False,1.0,2007.0,1,5,9
3,2181286,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,1.0,2007.0,1,5,19
4,2182838,2024,NARCOTICS,POSS: HEROIN(WHITE),STREET,True,False,19.0,2007.0,1,4,14


### 5 clusters

In [219]:
clusters2007_5 = km_5.fit_predict(year2007)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 171237, cost: 4637100.0
Run 1, iteration: 2/100, moves: 20756, cost: 4637099.0
Run 1, iteration: 3/100, moves: 0, cost: 4637099.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 174671, cost: 4446270.0
Run 2, iteration: 2/100, moves: 13674, cost: 4446270.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 184447, cost: 4522404.0
Run 3, iteration: 2/100, moves: 0, cost: 4522404.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 174230, cost: 4696952.0
Run 4, iteration: 2/100, moves: 6030, cost: 4696952.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 175236, cost: 4446598.0
Run 5, iteration: 2/100, moves: 19937, cost: 4446598.0

In [222]:
year2007_5clusters = pd.DataFrame(clusters2007_5)

In [224]:
year2007_5clusters.columns = ['clusters_5']

In [225]:
year2007New5 = pd.concat([year2007, year2007_5clusters], axis = 1)

In [227]:
year2007New5.to_csv("year2007_5clusters.csv")

In [228]:
year2007_5clusterCentroids = pd.DataFrame(km_5.cluster_centroids_)

In [229]:
year2007_5clusterCentroids.columns = year2007.columns

In [230]:
year2007_5clusterCentroids.to_csv("year2007_5clusterCentroids.csv")

### 10 clusters

In [220]:
clusters2007_10 = km_10.fit_predict(year2007)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 208028, cost: 4187183.0
Run 1, iteration: 2/100, moves: 21641, cost: 4173450.0
Run 1, iteration: 3/100, moves: 12200, cost: 4173450.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 177177, cost: 4262432.0
Run 2, iteration: 2/100, moves: 16379, cost: 4259982.0
Run 2, iteration: 3/100, moves: 1325, cost: 4259982.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 145327, cost: 4143374.0
Run 3, iteration: 2/100, moves: 56391, cost: 4134374.0
Run 3, iteration: 3/100, moves: 3368, cost: 4134374.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 164715, cost: 4304483.0
Run 4, iteration: 2/100, moves: 25691, cost: 4304482.0
Run 4, iteration: 3/100, moves: 0, cost: 4304482.0
Init: initializing cen

In [231]:
year2007_10clusters = pd.DataFrame(clusters2007_10)

In [232]:
year2007_10clusters.columns = ['clusters_10']

In [234]:
year2007New10 = pd.concat([year2007, year2007_10clusters], axis = 1)

In [236]:
year2007New10.to_csv("year2007_10clusters.csv")

In [238]:
year2007_10clusterCentroids = pd.DataFrame(km_10.cluster_centroids_)

In [239]:
year2007_10clusterCentroids.columns = year2007.columns

In [240]:
year2007_10clusterCentroids.to_csv("year2007_10clusterCentroids.csv")

### 25 clusters

In [221]:
clusters2007_25 = km_25.fit_predict(year2007)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 193622, cost: 3756223.0
Run 1, iteration: 2/100, moves: 40616, cost: 3754872.0
Run 1, iteration: 3/100, moves: 181, cost: 3754872.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 137328, cost: 3829491.0
Run 2, iteration: 2/100, moves: 26701, cost: 3825795.0
Run 2, iteration: 3/100, moves: 128, cost: 3825795.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 142210, cost: 3806101.0
Run 3, iteration: 2/100, moves: 13331, cost: 3806099.0
Run 3, iteration: 3/100, moves: 0, cost: 3806099.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 134089, cost: 3915971.0
Run 4, iteration: 2/100, moves: 40327, cost: 3915970.0
Run 4, iteration: 3/100, moves: 0, cost: 3915970.0
Init: initializing centroids

In [241]:
year2007_25clusters = pd.DataFrame(clusters2007_25)

In [242]:
year2007_25clusters.columns = ['clusters_25']

In [243]:
year2007New25 = pd.concat([year2007, year2007_25clusters], axis = 1)

In [245]:
year2007New25.to_csv("year2007_25clusters.csv")

In [246]:
year2007_25clusterCentroids = pd.DataFrame(km_25.cluster_centroids_)

In [247]:
year2007_25clusterCentroids.columns = year2007.columns

In [248]:
year2007_25clusterCentroids.to_csv("year2007_25clusterCentroids.csv")

In [249]:
year2007_25clusterCentroids

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour
0,2191307,0810,THEFT,OVER $500,STREET,False,False,12.0,2007.0,9,17,9
1,2240523,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,17.0,2007.0,3,25,1
2,2203809,0810,THEFT,OVER $500,STREET,False,False,1.0,2007.0,8,3,18
3,2240040,0460,BATTERY,SIMPLE,STREET,True,False,8.0,2007.0,3,27,11
4,2190460,0460,BATTERY,SIMPLE,RESIDENCE,False,False,7.0,2007.0,4,16,22
5,2227430,0610,BURGLARY,FORCIBLE ENTRY,APARTMENT,False,False,2.0,2007.0,7,29,11
6,2251686,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,False,4.0,2007.0,4,19,12
7,2194256,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,25.0,2007.0,1,17,23
8,2251229,0560,ASSAULT,SIMPLE,SIDEWALK,True,False,16.0,2007.0,7,29,20
9,2253485,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,8.0,2007.0,5,6,15


## YEAR 2008

In [250]:
year2008 = filterYear(2008)

In [251]:
year2008 = year2008.reset_index()

In [254]:
year2008.head()

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour
0,3795857,0110,HOMICIDE,FIRST DEGREE MURDER,ALLEY,True,False,3.0,2008.0,10,7,12
1,3795858,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,15.0,2008.0,10,9,3
2,3795859,0110,HOMICIDE,FIRST DEGREE MURDER,PARK PROPERTY,False,False,8.0,2008.0,10,9,8
3,3795860,0110,HOMICIDE,FIRST DEGREE MURDER,RESTAURANT,False,False,15.0,2008.0,10,10,2
4,3795861,0110,HOMICIDE,FIRST DEGREE MURDER,GARAGE,False,True,10.0,2008.0,10,10,12


### 5 clusters

In [256]:
clusters2008_5 = km_5.fit_predict(year2008)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 201497, cost: 6259938.0
Run 1, iteration: 2/100, moves: 26942, cost: 6255156.0
Run 1, iteration: 3/100, moves: 4, cost: 6255156.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 180958, cost: 6438973.0
Run 2, iteration: 2/100, moves: 23734, cost: 6438973.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 98523, cost: 6250622.0
Run 3, iteration: 2/100, moves: 24791, cost: 6247756.0
Run 3, iteration: 3/100, moves: 217, cost: 6247756.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 169376, cost: 6206952.0
Run 4, iteration: 2/100, moves: 36968, cost: 6206952.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 213377, cost: 6257473

In [259]:
year2008_5clusters = pd.DataFrame(clusters2008_5)

In [260]:
year2008_5clusters.columns = ['clusters_5']

In [262]:
year2008New5 = pd.concat([year2008, year2008_5clusters], axis = 1)

In [264]:
year2008New5.to_csv("year2008_5clusters.csv")

In [265]:
year2008_5clusterCentroids = pd.DataFrame(km_5.cluster_centroids_)

In [266]:
year2008_5clusterCentroids.columns = year2008.columns

In [268]:
year2008_5clusterCentroids.to_csv("year2008_5clusterCentroids.csv")

### 10 clusters

In [257]:
clusters2008_10 = km_10.fit_predict(year2008)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 157951, cost: 5743854.0
Run 1, iteration: 2/100, moves: 15630, cost: 5743852.0
Run 1, iteration: 3/100, moves: 0, cost: 5743852.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 176854, cost: 5655494.0
Run 2, iteration: 2/100, moves: 64622, cost: 5648107.0
Run 2, iteration: 3/100, moves: 4161, cost: 5648107.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 197143, cost: 5787220.0
Run 3, iteration: 2/100, moves: 74031, cost: 5768001.0
Run 3, iteration: 3/100, moves: 11908, cost: 5768000.0
Run 3, iteration: 4/100, moves: 0, cost: 5768000.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 261706, cost: 5778695.0
Run 4, iteration: 2/100, moves: 22906, cost: 5778695.0
Init: initializing centro

In [269]:
year2008_10clusters = pd.DataFrame(clusters2008_10)

In [270]:
year2008_10clusters.columns = ['clusters_10']

In [271]:
year2008New10 = pd.concat([year2008, year2008_10clusters], axis = 1)

In [273]:
year2008New10.to_csv("year2008_10clusters.csv")

In [274]:
year2008_10clusterCentroids = pd.DataFrame(km_10.cluster_centroids_)

In [275]:
year2008_10clusterCentroids.columns = year2008.columns

In [277]:
year2008_10clusterCentroids.to_csv("year2008_10clusterCentroids.csv")

### 25 clusters

In [258]:
clusters2008_25 = km_25.fit_predict(year2008)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 180435, cost: 5298640.0
Run 1, iteration: 2/100, moves: 10040, cost: 5298640.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 158258, cost: 5316142.0
Run 2, iteration: 2/100, moves: 19619, cost: 5295503.0
Run 2, iteration: 3/100, moves: 1630, cost: 5295503.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 117660, cost: 5391816.0
Run 3, iteration: 2/100, moves: 0, cost: 5391816.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 140906, cost: 5129119.0
Run 4, iteration: 2/100, moves: 40773, cost: 5126567.0
Run 4, iteration: 3/100, moves: 2700, cost: 5126567.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 213489, cost: 523777

In [278]:
year2008_25clusters = pd.DataFrame(clusters2008_25)

In [279]:
year2008_25clusters.columns = ['clusters_25']

In [280]:
year2008New25 = pd.concat([year2008, year2008_25clusters], axis = 1)

In [283]:
year2008New25.to_csv("year2008_25clusters.csv")

In [284]:
year2008_25clusterCentroids = pd.DataFrame(km_25.cluster_centroids_)

In [285]:
year2008_25clusterCentroids.columns = year2008.columns

In [287]:
year2008_25clusterCentroids.to_csv("year2008_25clusterCentroids.csv")

## YEAR 2009


In [288]:
year2009 = filterYear(2009)

In [289]:
year2009 = year2009.reset_index()

In [290]:
year2009.head()

,index,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour
0,3795934,1562,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,RESIDENCE,True,False,25.0,2009.0,1,1,0
1,3795964,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,12.0,2009.0,1,1,5
2,3795965,0110,HOMICIDE,FIRST DEGREE MURDER,ALLEY,False,False,22.0,2009.0,1,3,4
3,3795966,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,3.0,2009.0,1,3,18
4,3795967,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,4.0,2009.0,1,4,16


### 5 clusters

In [ ]:
clusters2009_5 = km_5.fit_predict(year2009)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 184328, cost: 5846243.0
Run 1, iteration: 2/100, moves: 9783, cost: 5846243.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 235109, cost: 5744605.0
Run 2, iteration: 2/100, moves: 24638, cost: 5744604.0
Run 2, iteration: 3/100, moves: 0, cost: 5744604.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 100081, cost: 5790883.0
Run 3, iteration: 2/100, moves: 64145, cost: 5773990.0


### 10 clusters

In [ ]:
clusters2009_10 = km_10.fit_predict(year2009)

### 25 clusters

In [ ]:
clusters2009_25 = km_25.fit_predict(year2009)